In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [8]:
x_data = obtain_SWNUNetwork_input(
    method = "umap",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings.numpy(),
    k=5,
    time_feature='time_encoding',
    standardise_method=None,
    add_time_in_path=False)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:12<00:00, 458.05it/s]


[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 415])

SWNU Network

Obtaining path by looking at post history
We can obtain a path by looking at the history of each post. Here we look at the last 20 posts (and pad with vectors of zeros if there are less than 20 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [7]:
#dimensionality reduction
embedding_dim = sbert_embeddings.shape[1]
dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
dimensions = [30]#[50,30,15]
#time features
time_features = "time_encoding"
standardise_method = "standardise"
add_time_in_path = False
#SWNU block
augmentation_tp = "Conv1d"
hidden_dim_aug = None
comb_m = "concatenation"
log_sig = True
conv_output_channels = [10] #[20, 10, 5]
log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
bidirectional = False
#ffn
hidden_dim_sizes = [54]#[32,64]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
#overall training
num_epochs = 100
patience = 4
learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [8]:
size=20
swnu_network_log_signature, best_swnu_network_log_signature, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dim_reduce_method,
    dimensions=dimensions, 
    sig_depths=[x[1] for x in log_signature_dimensions_and_sig_depths], 
    log_signature=log_sig,
    conv_output_channels=conv_output_channels,
    swnu_hidden_dim_sizes=[x[0] for x in log_signature_dimensions_and_sig_depths],
    ffn_hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    time_feature=time_features, 
    standardise_method=standardise_method,
    add_time_in_path = add_time_in_path,
    augmentation_type = augmentation_tp,
    hidden_dim_aug = hidden_dim_aug,
    comb_method = comb_m,
    path_indices=None,
    data_split_seed=123,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]



##################################################
dimension: 30 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.






































































































































































































































































































































































100%|██████████| 5568/5568 [00:19<00:00, 284.55it/s]


[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


Accuracy on dataset of size 891: 71.71717286109924 %.
Average loss: 0.1950046909706933
- f1: [0.75769231 0.66037736]
- f1 (macro): 0.7090348330914369


  0%|          | 0/1 [02:23<?, ?it/s]


KeyboardInterrupt: 